<a href="https://colab.research.google.com/github/mrSaggio/NEIRO/blob/main/Classifikator_Torch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/all_truba_digits'))

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)
#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
#preprocess.
from keras.preprocessing.image import ImageDataGenerator
#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from zipfile import ZipFile
from PIL import Image

In [ ]:
X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR0='/content/drive/MyDrive/all_truba_digits/0'
FLOWER_SUNFLOWER_DIR1='/content/drive/MyDrive/all_truba_digits/1'
FLOWER_TULIP_DIR2='/content/drive/MyDrive/all_truba_digits/2'
FLOWER_DANDI_DIR3='/content/drive/MyDrive/all_truba_digits/3'
FLOWER_ROSE_DIR4='//content/drive/MyDrive/all_truba_digits/4'
FLOWER_ROSE_DIR5='//content/drive/MyDrive/all_truba_digits/5'
FLOWER_ROSE_DIR6='//content/drive/MyDrive/all_truba_digits/6'
FLOWER_ROSE_DIR7='//content/drive/MyDrive/all_truba_digits/7'
FLOWER_ROSE_DIR8='//content/drive/MyDrive/all_truba_digits/8'
FLOWER_ROSE_DIR9='//content/drive/MyDrive/all_truba_digits/9'
FLOWER_ROSE_DIRS='//content/drive/MyDrive/all_truba_digits/Symbol'


In [ ]:
def assign_label(img,flower_type):
    return flower_type



In [ ]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        X.append(np.array(img))
        Z.append(str(label))


In [ ]:
make_train_data('0',FLOWER_DAISY_DIR0)
make_train_data('1',FLOWER_DAISY_DIR1)
make_train_data('2',FLOWER_DAISY_DIR2)
make_train_data('3',FLOWER_DAISY_DIR3)
make_train_data('4',FLOWER_DAISY_DIR4)
make_train_data('5',FLOWER_DAISY_DIR5)
make_train_data('6',FLOWER_DAISY_DIR6)
make_train_data('7',FLOWER_DAISY_DIR7)
make_train_data('8',FLOWER_DAISY_DIR8)
make_train_data('9',FLOWER_DAISY_DIR9)
make_train_data('Symbol',FLOWER_DAISY_DIRS)
l=print(list.make_train_data)
print(len(X))

In [ ]:
fig,ax=plt.subplots(11,4)
fig.set_size_inches(15,15)
for i in range(11):
    for j in range (4):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Алфавит='+Z[l])

plt.tight_layout()



In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [ ]:
batch_size=128
epochs=50

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#History = model.fit_generator(x_train,y_train,
                               #validation_data = (x_test,y_test),
                              #verbose = 1)
History = model.fit(x_train,y_train,validation_data = (x_test,y_test),epochs = 5)

In [ ]:
History.history

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
prop_class

In [ ]:
mis_class

In [ ]:
plt.imshow(x_test[1])

In [ ]:
np.argmax(y_test[1],axis = 0)

In [ ]:
pred_digits[1]

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str([pred_digits[prop_class[count]]])+"\n"+"Actual Flower : "+str(np.argmax(y_test[prop_class[count]])))
        plt.tight_layout()
        count+=1

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])

        ax[i,j].set_title("Predicted Flower : "+str([pred_digits[mis_class[count]]])+"\n"+"Actual Flower : "+str(np.argmax(y_test[mis_class[count]])))
        plt.tight_layout()
        count+=1